In [1]:
!pip install pulser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
      Successfully uninstalled idna-3.4
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.18 requires requests>=2.26, but you have requests 2.25.1 which is incompatible.
tweepy 4.1

In [2]:
import numpy as np
from scipy.optimize import minimize
from pulser import Sequence, Register, Pulse
from pulser.devices import Chadoq2
from pulser_simulation import Simulation
from gate_pulses import u3_pulse

def composition(u, v):
    def func(x):
        return v(u(x))
    return func

def get_proba(N_samples=2048):
    def func(v):
        gamma, theta, phi = v
        reg = Register({"q0": (0,0)})
        seq = Sequence(reg, Chadoq2)
        seq.declare_channel("raman", "raman_local", initial_target="q0")
        seq.add(u3_pulse(gamma, theta, phi), "raman")
        seq.measure(basis="digital")
        sim = Simulation(seq)
        res = sim.run()
        states = res.sample_final_state(N_samples=N_samples)

        return states['0']/N_samples, states['1']/N_samples
    return func


def mae(x):
    def func(y):
        return np.linalg.norm(np.array(x) - np.array(y), 1)
    return func

if __name__ == "__main__":
    p0 = np.random.random((2,))
    p0 = p0 / np.linalg.norm(p0, 1)
    #p0 = np.array([0.51357006, .4864298094])
    res = minimize(
            composition(get_proba(), mae(p0)),
            x0=2*np.pi*np.random.random((3,)),
            method="COBYLA",
            tol=1e-4,
            options={"maxiter": 500}
            )

    print(f'p0: {p0}\nproba: {get_proba()(res.x)}')

p0: [0.40472642 0.59527358]
proba: (0.42041015625, 0.57958984375)
